In [1]:
%%javascript
// builds the TOC
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# A Decision Tree of Observable Operators

> Part 1: NEW Observables.
> source: http://reactivex.io/documentation/operators.html#tree.  
> (transcribed to RxPY 1.5.7, Py2.7 / 2016-12, Gunther Klessinger, [axiros](http://www.axiros.com))  

**This tree can help you find the ReactiveX Observable operator you’re looking for.**  

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

## Usage

There are no configured behind the scenes imports or code, all code is explicitly given in the notebook.  
All initialisiation of tools is in the first python code cell, meaning you always have to run the first cell after ipython kernel restarts.  
**All other cells are autonmous.**

In the use case functions, in contrast to the official examples we simply use **`rand`** (mapped to `randint(0, 100)`) a lot, to demonstrate when/how often observable sequences are generated and when their result is buffered for various subscribers.  
*When in doubt then run the cell again, you might have been "lucky" and got the same random.*

### RxJS
The functions are linked to the [official documentation](http://reactivex.io/documentation/operators.html#tree) and created roughly analogous to the **RxJS** examples.  

### Output
When the output is not in marble format we display it like so:

```
new subscription on stream 276507289 

   3.4 [next]    1.4: {'answer': 48}
   3.5 [cmpl]    1.6: fin
   
```
where the lines are syncronously `print`ed as they happen.  
For each use case in `reset_start_time()` (alias `rst`), a global timer is set to 0 and we show the offset to it, in *milliseconds* & with one decimal value and also the offset to the start of stream subscription. In the example 3.4, 3.5 are millis since global counter reset, while 1.4, 1.6 are offsets to start of subscription.




In [44]:
# Helpers.
# Run this cell always after kernel restarts. All other cells are autonomous.
from __future__ import print_function
import rx
import time
from random import randint
from rx.testing import marbles
sleep, now = time.sleep, time.time
O = rx.Observable

ts_glob = 0 # global start time
def reset_start_time(show_doc_for=None, title=None):
    'resets global start time and also prints doc strings'
    global ts_glob
    ts_glob, d = time.time(), show_doc_for
    if title:
        header(title)
    if not d:
        return
    # print the function doc if given
    print ('%s:\n%s\n%s' % (d.__module__, d.__doc__ or 'n.a.', '-' * 80))
        
rst = reset_start_time

def log(*msg):
    s = ' '.join([str(s) for s in msg])
    print ('%s %s' % (dt(ts_glob), s))

def header(msg):
    print ('\n\n%s %s %s\n' % ('=' * 10, msg, '=' * 10))
    
def rand():
    return randint(0, 100)

def to_int(s):
    return int(s) if s.isdigit() else s

def dt(ts):
    # the time delta of now to given ts (in millis, 1 float)
    return str('%.1f' % ((time.time() - ts) * 1000)).rjust(6)

class ItemGetter:
    'allows to throw an object onto a format string'
    def __init__(self, obj):
        self.obj = obj
    def __getitem__(self, k, d=None):
        return getattr(self.obj, k, d)

class Subscriber:
    def __init__(self, observed_stream, **kw):
        print ('')
        log('New subscription on stream', hash(observed_stream))
        self.ts = time.time() # tstart, for dts at events
        self.name = ' (%(name)s)' % kw if 'name' in kw else ''
        
    def _on(self, what, v=''):
        print ('%s [%s] %s: %s%s' % (
                dt(ts_glob), what, dt(self.ts), v, self.name))

    def on_next     (self, v): return self._on('next', v)
    def on_error    (self, v): return self._on('err ', v)
    def on_completed(self)   : return self._on('cmpl', 'fin')
    
def subs(src, **kw):
    return src.subscribe(Subscriber(src, **kw))

# I want to create a **NEW** Observable...

## ... that emits a particular item: **[just](http://reactivex.io/documentation/operators/just.html) **

In [45]:
reset_start_time(O.just)
stream = O.just({'answer': rand()})
disposable = subs(stream)
sleep(0.5)
disposable = subs(stream) # same answer
# all stream ops work, its a real stream:
disposable = subs(stream.map(lambda x: x.get('answer', 0) * 2))

rx.linq.observable.returnvalue:
Returns an observable sequence that contains a single element,
    using the specified scheduler to send out observer messages.
    There is an alias called 'just'.

    example
    res = rx.Observable.return(42)
    res = rx.Observable.return(42, rx.Scheduler.timeout)

    Keyword arguments:
    value -- Single element in the resulting observable sequence.
    scheduler -- [Optional] Scheduler to send the single element on. If
        not specified, defaults to Scheduler.immediate.

    Returns an observable sequence containing the single specified
    element.
    
--------------------------------------------------------------------------------

   0.7 New subscription on stream 276064481
   1.3 [next]    0.5: {'answer': 0}
   1.7 [cmpl]    1.0: fin

 502.6 New subscription on stream 276064481
 503.3 [next]    0.4: {'answer': 0}
 503.4 [cmpl]    0.5: fin

 503.9 New subscription on stream 276501117
 504.8 [next]    0.8: 0
 505.1 [cmpl]    1.2: fin


## ..that was returned from a function *called at subscribe-time*: **[start](http://reactivex.io/documentation/operators/start.html)**

In [18]:
print('There is a little API difference to RxJS, see Remarks:\n')
rst(O.start)

def f():
    log('function called')
    return rand()

stream = O.start(func=f)
d = subs(stream)
d = subs(stream)

header("Exceptions are handled correctly (an observable should never except):")

def breaking_f():    
    return 1 / 0

stream = O.start(func=breaking_f)
d = subs(stream)
d = subs(stream)



# startasync: only in python3 and possibly here(?) http://www.tornadoweb.org/en/stable/concurrent.html#tornado.concurrent.Future
#stream = O.start_async(f)
#d = subs(stream)


There is a little API difference to RxJS, see Remarks:

rx.linq.observable.start:
Invokes the specified function asynchronously on the specified
    scheduler, surfacing the result through an observable sequence.

    Example:
    res = rx.Observable.start(lambda: pprint('hello'))
    res = rx.Observable.start(lambda: pprint('hello'), rx.Scheduler.timeout)

    Keyword arguments:
    func -- {Function} Function to run asynchronously.
    scheduler -- {Scheduler} [Optional] Scheduler to run the function on. If
        not specified, defaults to Scheduler.timeout.

    Returns {Observable} An observable sequence exposing the function's
    result value, or an exception.

    Remarks:
    The function is called immediately, not during the subscription of the
    resulting sequence. Multiple subscriptions to the resulting sequence can
    observe the function's result.
    
--------------------------------------------------------------------------------
   1.3 function called

   1.0 New s

## ..that was returned from an Action, Callable, Runnable, or something of that sort, called at subscribe-time: **[from](http://reactivex.io/documentation/operators/from.html)**

In [43]:
rst(O.from_iterable)
def f():
    log('function called')
    return rand()
# aliases: O.from_, O.from_list
# 1.: From a tuple:
stream = O.from_iterable((1,2,rand()))
d = subs(stream)
# d = subs(stream) # same result

# 2. from a generator
gen = (rand() for j in range(3))
stream = O.from_iterable(gen)
d = subs(stream)



rx.linq.observable.fromiterable:
Converts an array to an observable sequence, using an optional
    scheduler to enumerate the array.

    1 - res = rx.Observable.from_iterable([1,2,3])
    2 - res = rx.Observable.from_iterable([1,2,3], rx.Scheduler.timeout)

    Keyword arguments:
    :param Observable cls: Observable class
    :param Scheduler scheduler: [Optional] Scheduler to run the
        enumeration of the input sequence on.

    :returns: The observable sequence whose elements are pulled from the
        given enumerable sequence.
    :rtype: Observable
    
--------------------------------------------------------------------------------

   0.8 New subscription on stream 276467533
   1.2 [next]    0.4: 1
   1.4 [next]    0.6: 2
   1.7 [next]    0.9: 81
   2.2 [cmpl]    1.4: fin

   2.9 New subscription on stream 276467469
   3.3 [next]    0.3: 22
   3.5 [next]    0.5: 65
   3.7 [next]    0.7: 72
   3.8 [cmpl]    0.8: fin


In [44]:
rst(O.from_callback)
# in my words: In the on_next of the subscriber you'll have the original arguments,
# potentially objects, e.g. user original http requests.
# i.e. you could merge those with the result stream of a backend call to
# a webservice or db and send the request.response back to the user then.

def g(f, a, b):
    f(a, b)
    log('called f')
stream = O.from_callback(lambda a, b, f: g(f, a, b))('fu', 'bar')
d = subs(stream.delay(200))
# d = subs(stream.delay(200)) # does NOT work


rx.linq.observable.fromcallback:
Converts a callback function to an observable sequence.

    Keyword arguments:
    func -- {Function} Function with a callback as the last parameter to
        convert to an Observable sequence.
    selector -- {Function} [Optional] A selector which takes the arguments
        from the callback to produce a single item to yield on next.

    Returns {Function} A function, when executed with the required
    parameters minus the callback, produces an Observable sequence with a
    single value of the arguments to the callback as a list.
    
--------------------------------------------------------------------------------

   0.7 New subscription on stream 276467521
   1.8 called f
 205.0 [next]  204.2: ['fu', 'bar']
 205.4 [cmpl]  204.5: fin


## ...after a specified delay: **[timer](http://reactivex.io/documentation/operators/timer.html)**

In [45]:
rst()
# start a stream of 0, 1, 2, .. after 200 ms, with a delay of 100 ms:
stream = O.timer(200, 100).time_interval()\
    .map(lambda x: 'val:%s dt:%s' % (x.value, x.interval))\
    .take(3)
d = subs(stream, name='observer1')
# intermix directly with another one
d = subs(stream, name='observer2')


   0.7 New subscription on stream 276459573

   2.2 New subscription on stream 276459573
 205.6 [next]  204.9: val:0 dt:0:00:00.204346 (observer1)
 206.8 [next]  204.6: val:0 dt:0:00:00.204324 (observer2)
 305.8 [next]  305.0: val:1 dt:0:00:00.100137 (observer1)
 308.5 [next]  306.3: val:1 dt:0:00:00.101737 (observer2)
 406.0 [next]  405.3: val:2 dt:0:00:00.100271 (observer1)
 406.4 [cmpl]  405.6: fin (observer1)
 407.1 [next]  404.9: val:2 dt:0:00:00.098561 (observer2)
 408.5 [cmpl]  406.3: fin (observer2)


## ...that emits a sequence of items repeatedly: **[repeat](http://reactivex.io/documentation/operators/repeat.html) **

In [27]:
rst(O.repeat)
# repeat is over *values*, not function calls. Use generate or create for function calls!
subs(O.repeat({'rand': time.time()}, 3))

header('do while:')
l = []
def condition(x):
    l.append(1)
    return True if len(l) < 2 else False
stream = O.just(42).do_while(condition)
d = subs(stream)



rx.linq.observable.repeat:
Generates an observable sequence that repeats the given element the
    specified number of times, using the specified scheduler to send out
    observer messages.

    1 - res = rx.Observable.repeat(42)
    2 - res = rx.Observable.repeat(42, 4)
    3 - res = rx.Observable.repeat(42, 4, Rx.Scheduler.timeout)
    4 - res = rx.Observable.repeat(42, None, Rx.Scheduler.timeout)

    Keyword arguments:
    value -- Element to repeat.
    repeat_count -- [Optional] Number of times to repeat the element. If not
        specified, repeats indefinitely.
    scheduler -- Scheduler to run the producer loop on. If not specified,
        defaults to ImmediateScheduler.

    Returns an observable sequence that repeats the given element the
    specified number of times.
--------------------------------------------------------------------------------

   0.6 New subscription on stream 276524669
   1.3 [next]    0.4: {'rand': 1482182963.392313}
   1.7 [next]    0.8: {'rand':

## ...from scratch, with custom logic and cleanup (calling a function again and again): **[create](http://reactivex.io/documentation/operators/create.html) **

In [51]:
rx = O.create
rst(rx)

def f(obs):
    # this function is called for every observer
    obs.on_next(rand())
    obs.on_next(rand())
    obs.on_completed()
    def cleanup():
        log('cleaning up...')
    return cleanup
stream = O.create(f).delay(200) # the delay causes the cleanup called before the subs gets the vals
d = subs(stream)
d = subs(stream)




sleep(0.5)
rst(title='Exceptions are handled nicely')
l = []
def excepting_f(obs):
    for i in range(3):
        l.append(1)
        obs.on_next('%s %s (observer hash: %s)' % (i, 1. / (3 - len(l)), hash(obs) ))
    obs.on_completed()

stream = O.create(excepting_f)
d = subs(stream)
d = subs(stream)




rst(title='Feature or Bug?')
print('(where are the first two values?)')
l = []
def excepting_f(obs):
    for i in range(3):
        l.append(1)
        obs.on_next('%s %s (observer hash: %s)' % (i, 1. / (3 - len(l)), hash(obs) ))
    obs.on_completed()

stream = O.create(excepting_f).delay(100)
d = subs(stream)
d = subs(stream)
# I think its an (amazing) feature, preventing to process functions results of later(!) failing functions


rx.linq.observable.create:
n.a.
--------------------------------------------------------------------------------

   0.8 New subscription on stream 276492845
   1.7 cleaning up...

   2.0 New subscription on stream 276492845
   3.7 cleaning up...
 207.2 [next]  206.3: 52
 208.5 [next]  206.5: 19 207.5 [next]  206.7: 34

 209.0 [next]  207.0: 17 208.1 [cmpl]  207.2: fin

 209.1 [cmpl]  207.1: fin


========== Exceptions are handled nicely ==========


   1.3 New subscription on stream 276412933
   1.6 [next]    0.2: 0 0.5 (observer hash: 276472189)
   2.0 [next]    0.6: 1 1.0 (observer hash: 276472189)
   2.3 [err ]    1.0: float division by zero

   3.1 New subscription on stream 276412933
   3.6 [next]    0.2: 0 -1.0 (observer hash: 275126105)
   3.9 [next]    0.5: 1 -0.5 (observer hash: 275126105)
   4.1 [next]    0.7: 2 -0.333333333333 (observer hash: 275126105)
   4.2 [cmpl]    0.8: fin


========== Feature or Bug? ==========

(where are the first two values?)

   1.1 New subscript

In [9]:
rx = O.generate
rst(rx)
"""The basic form of generate takes four parameters:

the first item to emit
a function to test an item to determine whether to emit it (true) or terminate the Observable (false)
a function to generate the next item to test and emit based on the value of the previous item
a function to transform items before emitting them
"""
def generator_based_on_previous(x): return x + 1.1
def doubler(x): return 2 * x
d = subs(rx(0, lambda x: x < 4, generator_based_on_previous, doubler))

rx.linq.observable.generate:
Generates an observable sequence by running a state-driven loop
    producing the sequence's elements, using the specified scheduler to
    send out observer messages.

    1 - res = rx.Observable.generate(0,
        lambda x: x < 10,
        lambda x: x + 1,
        lambda x: x)
    2 - res = rx.Observable.generate(0,
        lambda x: x < 10,
        lambda x: x + 1,
        lambda x: x,
        Rx.Scheduler.timeout)

    Keyword arguments:
    initial_state -- Initial state.
    condition -- Condition to terminate generation (upon returning False).
    iterate -- Iteration step function.
    result_selector -- Selector function for results produced in the
        sequence.
    scheduler -- [Optional] Scheduler on which to run the generator loop.
        If not provided, defaults to CurrentThreadScheduler.

    Returns the generated sequence.
    
--------------------------------------------------------------------------------

   0.8 New subscription on 

In [49]:
rx = O.generate_with_relative_time
rst(rx)
stream = rx(1, lambda x: x < 4, lambda x: x + 1, lambda x: x, lambda t: 100)
d = subs(stream)


rx.linq.observable.generatewithrelativetime:
Generates an observable sequence by iterating a state from an
    initial state until the condition fails.

    res = source.generate_with_relative_time(0,
        lambda x: True,
        lambda x: x + 1,
        lambda x: x,
        lambda x: 500)

    initial_state -- Initial state.
    condition -- Condition to terminate generation (upon returning false).
    iterate -- Iteration step function.
    result_selector -- Selector function for results produced in the
        sequence.
    time_selector -- Time selector function to control the speed of values
        being produced each iteration, returning integer values denoting
        milliseconds.
    scheduler -- [Optional] Scheduler on which to run the generator loop.
        If not specified, the timeout scheduler is used.

    Returns the generated sequence.
    
--------------------------------------------------------------------------------

   0.9 New subscription on stream 27607062

## ...for each observer that subscribes OR according to a condition at subscription time: **[defer / if_then](http://reactivex.io/documentation/operators/defer.html) **

In [53]:
rst(O.defer)
# plural! (unique per subscription)
streams = O.defer(lambda: O.just(rand()))
d = subs(streams)
d = subs(streams) # gets other values - created by subscription!

rx.linq.observable.defer:
Returns an observable sequence that invokes the specified factory
    function whenever a new observer subscribes.

    Example:
    1 - res = rx.Observable.defer(lambda: rx.Observable.from_([1,2,3]))

    Keyword arguments:
    :param types.FunctionType observable_factory: Observable factory function
        to invoke for each observer that subscribes to the resulting sequence.

    :returns: An observable sequence whose observers trigger an invocation
    of the given observable factory function.
    :rtype: Observable
    
--------------------------------------------------------------------------------

   0.7 New subscription on stream 276459581
   1.1 [next]    0.3: 67
   1.3 [cmpl]    0.5: fin

   1.9 New subscription on stream 276459581
   2.4 [next]    0.4: 91
   2.5 [cmpl]    0.5: fin


In [55]:
# evaluating a condition at subscription time in order to decide which of two streams to take.
rst(O.if_then)
cond = True
def should_run():
    return cond
streams = O.if_then(should_run, O.return_value(43), O.return_value(56))
d = subs(streams)

log('condition will now evaluate falsy:')
cond = False
streams = O.if_then(should_run, O.return_value(43), O.return_value(rand()))
d = subs(streams)
d = subs(streams)

rx.linq.observable.ifthen:
Determines whether an observable collection contains values.

    Example:
    1 - res = rx.Observable.if(condition, obs1)
    2 - res = rx.Observable.if(condition, obs1, obs2)
    3 - res = rx.Observable.if(condition, obs1, scheduler=scheduler)

    Keyword parameters:
    condition -- {Function} The condition which determines if the
        then_source or else_source will be run.
    then_source -- {Observable} The observable sequence or Promise that
        will be run if the condition function returns true.
    else_source -- {Observable} [Optional] The observable sequence or
        Promise that will be run if the condition function returns False.
        If this is not provided, it defaults to rx.Observable.empty
    scheduler -- [Optional] Scheduler to use.

    Returns an observable {Observable} sequence which is either the
    then_source or else_source.
    
--------------------------------------------------------------------------------

   0.9 New

## ...that emits a sequence of integers: **[range](http://reactivex.io/documentation/operators/range.html) **

In [60]:
rst(O.range)
d = subs(O.range(0, 3))

rx.linq.observable.range:
Generates an observable sequence of integral numbers within a
    specified range, using the specified scheduler to send out observer
    messages.

    1 - res = Rx.Observable.range(0, 10)
    2 - res = Rx.Observable.range(0, 10, rx.Scheduler.timeout)

    Keyword arguments:
    start -- The value of the first integer in the sequence.
    count -- The number of sequential integers to generate.
    scheduler -- [Optional] Scheduler to run the generator loop on. If not
        specified, defaults to Scheduler.current_thread.

    Returns an observable sequence that contains a range of sequential
    integral numbers.
    
--------------------------------------------------------------------------------

   0.4 New subscription on stream 276462129
   1.0 [next]    0.5: 0
   1.6 [next]    1.1: 1
   2.1 [next]    1.6: 2
   2.5 [cmpl]    2.0: fin


### ...at particular intervals of time: **[interval](http://reactivex.io/documentation/operators/interval.html) **

In [78]:
rst(O.interval)
d = subs(O.interval(100).time_interval().map(lambda x, v: '%(interval)s %(value)s' % ItemGetter(x)).take(3))

rx.linq.observable.interval:
Returns an observable sequence that produces a value after each
    period.

    Example:
    1 - res = rx.Observable.interval(1000)
    2 - res = rx.Observable.interval(1000, rx.Scheduler.timeout)

    Keyword arguments:
    period -- Period for producing the values in the resulting sequence
        (specified as an integer denoting milliseconds).
    scheduler -- [Optional] Scheduler to run the timer on. If not specified,
        rx.Scheduler.timeout is used.

    Returns an observable sequence that produces a value after each period.
    
--------------------------------------------------------------------------------

   1.1 New subscription on stream 276538797
 103.0 [next]  101.7: 0:00:00.100954 0
 207.2 [next]  205.9: 0:00:00.104143 1
 310.3 [next]  309.0: 0:00:00.103126 2
 311.2 [cmpl]  309.9: fin


### ...after a specified delay (see timer)

## ...that completes without emitting items: **[empty](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [8]:
rst(O.empty)
d = subs(O.empty())

rx.linq.observable.empty:
Returns an empty observable sequence, using the specified scheduler
    to send out the single OnCompleted message.

    1 - res = rx.Observable.empty()
    2 - res = rx.Observable.empty(rx.Scheduler.timeout)

    scheduler -- Scheduler to send the termination call on.

    Returns an observable sequence with no elements.
    
--------------------------------------------------------------------------------

   0.6 New subscription on stream 276351057
   0.9 [cmpl]    0.2: fin


## ...that does nothing at all: **[never](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [7]:
rst(O.never)
d = subs(O.never())

rx.linq.observable.never:
Returns a non-terminating observable sequence, which can be used to
    denote an infinite duration (e.g. when using reactive joins).

    Returns an observable sequence whose observers will never get called.
    
--------------------------------------------------------------------------------

   1.0 New subscription on stream 276351133


## ...that excepts: **[throw](http://reactivex.io/documentation/operators/empty-never-throw.html) **

In [6]:
rst(O.throw)
d = subs(O.throw(ZeroDivisionError))

rx.linq.observable.throw:
Returns an observable sequence that terminates with an exception,
    using the specified scheduler to send out the single OnError message.

    1 - res = rx.Observable.throw_exception(Exception('Error'))
    2 - res = rx.Observable.throw_exception(Exception('Error'),
                                            rx.Scheduler.timeout)

    Keyword arguments:
    exception -- An object used for the sequence's termination.
    scheduler -- Scheduler to send the exceptional termination call on. If
        not specified, defaults to ImmediateScheduler.

    Returns the observable sequence that terminates exceptionally with the
    specified exception object.
    
--------------------------------------------------------------------------------

   0.8 New subscription on stream 276351081
   1.4 [err ]    0.3: <type 'exceptions.ZeroDivisionError'>
